In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import seaborn as sns
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau


In [ ]:

# Set seed
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)


In [ ]:
# Load metadata
df = pd.read_csv("C:\Skin Classification\HAM10000_images\HAM10000_metadata.csv")  # Adjust path as needed

# Image path
img_dir = "C:\Skin Classification\HAM10000_images"  # Folder with image files

# Load and preprocess images
def load_images(df, img_dir, img_size=224):
    images = []
    for file_id in df['image_id']:
        img_path = os.path.join(img_dir, file_id + ".jpg")
        img = cv2.imread(img_path)
        img = cv2.resize(img, (img_size, img_size))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        images.append(img)
    return np.array(images)


In [4]:

X = load_images(df, img_dir)
y = df['dx']

# Encode labels
le = LabelEncoder()
y_encoded = le.fit_transform(y)
y_cat = to_categorical(y_encoded)

# Train-test split (stratified)
X_train, X_test, y_train, y_test = train_test_split(X, y_cat, test_size=0.2, stratify=y_cat, random_state=SEED)



In [ ]:
# Normalize images
X_train = X_train / 255.0
X_test = X_test / 255.0

# Class weights to handle imbalance
class_weights = compute_class_weight('balanced', classes=np.unique(y_encoded), y=y_encoded)
class_weights_dict = {i: w for i, w in enumerate(class_weights)}


In [ ]:
# Normalize images
X_train = X_train / 255.0
X_test = X_test / 255.0

# Class weights to handle imbalance
class_weights = compute_class_weight('balanced', classes=np.unique(y_encoded), y=y_encoded)
class_weights_dict = {i: w for i, w in enumerate(class_weights)}



In [ ]:
# Data augmentation
train_aug = ImageDataGenerator(
    rotation_range=40,
    zoom_range=0.2,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
)

In [ ]:

# Build model with EfficientNetB0
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Freeze base
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.3)(x)
predictions = Dense(7, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Compile
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Callbacks
callbacks = [
    EarlyStopping(patience=5, restore_best_weights=True),
    ReduceLROnPlateau(patience=2)
]


In [ ]:
# Train
history = model.fit(
    train_aug.flow(X_train, y_train, batch_size=32),
    validation_data=(X_test, y_test),
    epochs=30,
    class_weight=class_weights_dict,
    callbacks=callbacks
)

In [ ]:
# Evaluate
loss, acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {acc:.4f}")

In [ ]:
# Save model
model.save("skin_lesion_classifier.h5")

In [ ]:

# Plot training
plt.plot(history.history['accuracy'], label='Train Acc')
plt.plot(history.history['val_accuracy'], label='Val Acc')
plt.legend()
plt.title("Training Accuracy Curve")
plt.show()